In [19]:
import pandas as pd
import numpy as np
import os
pd.set_option("display.max_columns", None)
import glob

from astroquery.sdss import SDSS, Conf
from astropy.config import set_temp_cache
from astropy.utils.data import clear_download_cache
from astropy.time import Time

In [57]:
DATA_DIR = '/astro/store/epyc3/data3/adam_datasets/sdss_dr9_all'

In [63]:
#filter to a tenth of a RA degree

#rework to include all bands - use pandas magic (split col into rows (pd.explode)) and then set obs_id to obs_id_(band)
ras = np.linspace(0, 360, 360 * 10 + 1)

#checks only photoobj - use photoobjall table to gather all photometric observations? try to check all bands instead of one?
def query_by_ra(ra_start, ra_stop, check_download_integrity=False):
    file_name = os.path.join(DATA_DIR, f"sdss_dr9_observations_{ra_start:06.2f}_{ra_stop:06.2f}_all.csv")
    
    if not os.path.exists(file_name):
        print(ra_start, ra_stop)
        query = f"""
                SELECT objID, fieldID, field, ra, dec, raErr, decErr, 
                u, err_u, TAI_u, 
                g, err_g, TAI_g, 
                r, err_r, TAI_r, 
                i, err_i, TAI_i,
                z, err_z, TAI_z
                FROM PhotoObj
                WHERE (ra >= {ra_start}) AND (ra < {ra_stop}) 
                """
        raw_data = SDSS.query_sql(query, timeout=3600, data_release=9).to_pandas()
        
        #formatted_data = {
        #                    'obs_id':raw_data['objID'].astype(str), # add _filter to tag
        #                    'exposure_id':raw_data['fieldID'].astype(str),
        #                    'mjd_utc':raw_data['TAI_i']/(24*3600),
        #                    'ra':raw_data['ra'],
        #                    'dec':raw_data['dec'],
        #                    'ra_sigma':raw_data['raErr'],
        #                    'dec_sigma':raw_data['decErr'],
        #                    'filter':'i',
        #                    'mag':raw_data['i'],
        #                    'mag_sigma':raw_data['err_i'],
        #                    'observatory_code':'645'
        #                }
        
        

        filters = ['u', 'g', 'r', 'i', 'z']
        columns = ['ra', 'dec', 'fieldID', 'objID', 'raErr', 'decErr', 'obs']
        df_list = []
        for f in filters:
            df = pd.DataFrame(columns=['obs_id', 'ra', 'dec', 'ra_sigma', 'dec_sigma', 
                                       'mag', 'mag_sigma', 'filter', 'observatory_code'])

            df['obs_id'] = raw_data['objID'].apply(lambda x: f'{x}_{f}')
            df['exposure_id'] = raw_data['fieldID']
            df['ra'] = raw_data['ra']
            df['ra_sigma'] = raw_data['raErr']
            df['dec'] = raw_data['dec']
            df['dec_sigma'] = raw_data['decErr']
            df['filter'] = f
            df['mag'] = raw_data[f]
            df['mag_sigma'] = raw_data[f'err_{f}']
            df['mjd_utc'] = Time(raw_data[f'TAI_{f}']/(24*3600), scale='tai', format='mjd').utc.mjd
            df['observatory_code'] = '645'
            df_list.append(df)

        df = pd.concat(df_list, ignore_index=True)
        print(f'before: {len(df)}')
        df = df[df['mag'] != -9999]
        print(f'after: {len(df)}')
        df.sort_values(by=['mjd_utc', 'exposure_id'], ignore_index=True)

        #results = pd.DataFrame(formatted_data)
        #results.to_csv(file_name, index=False)
        df.to_csv(file_name, index=False)
        clear_download_cache(pkgname='astropy')
        
        
    if check_download_integrity:

        downloaded_results = pd.read_csv(file_name, index_col=False)

        query = f"""
                SELECT COUNT(*) AS COUNT
                FROM PhotoObj
                WHERE (ra >= {ra_start}) AND (ra < {ra_stop})
                """
        results = SDSS.query_sql(query, timeout=3600, data_release=9)
        #print(results)

        n_results = results["COUNT"][0]
        n_downloaded_results = len(downloaded_results)

        if n_results != n_downloaded_results:
            err = (f"Downloaded file ({file_name}) contains {n_results} rows while query expected {n_downloaded_results} rows.")
            raise ValueError(err)

In [29]:
ra_start = 0
ra_stop = .1
#query_by_ra(ra_start, ra_stop)
query = f"""
                SELECT TOP 10 objID, fieldID, field, ra, dec, raErr, decErr, i, err_i, TAI_i, u, err_u, TAI_u, g, err_g, TAI_g
                FROM PhotoObj
                WHERE (ra >= {ra_start}) AND (ra < {ra_stop}) AND i != -9999
        """
raw_data = SDSS.query_sql(query, timeout=3600, data_release=9).to_pandas()

#df = pd.read_csv(os.path.join(DATA_DIR, f"sdss_dr9_observations_{ra_start:06.2f}_{ra_stop:06.2f}.csv"))

In [59]:
query_by_ra(0, .1)

0 0.1
before: 1254705
after: 1254616


In [51]:
df = pd.read_csv(os.path.join(DATA_DIR, f"sdss_dr9_observations_{ra_start:06.2f}_{ra_stop:06.2f}_all.csv"))

In [56]:
df['mjd_utc'].max() - df['mjd_utc'].min()

4077.000196759269

In [31]:
raw_data

,objID,fieldID,field,ra,dec,raErr,decErr,i,err_i,TAI_i,u,err_u,TAI_u,g,err_g,TAI_g
0,1237645876871561233,1237645876871561216,168,0.030843,-0.906976,0.066555,0.083157,20.235672,0.049130,4.412906e+09,24.106367,1.033302,4.412906e+09,23.350119,0.327600,4.412906e+09
1,1237645876871561348,1237645876871561216,168,0.030787,-0.855554,0.015622,0.013614,18.512777,0.012075,4.412906e+09,24.556908,1.083104,4.412906e+09,21.087315,0.044674,4.412906e+09
2,1237645876871561516,1237645876871561216,168,0.029963,-1.010292,0.208353,0.112413,20.370285,0.085961,4.412906e+09,22.791155,0.594999,4.412906e+09,21.964983,0.149627,4.412906e+09
3,1237645876871561575,1237645876871561216,168,0.076610,-1.028193,0.049008,0.061250,19.716581,0.040413,4.412906e+09,22.938829,0.579495,4.412906e+09,21.498795,0.081531,4.412906e+09
4,1237645876871561576,1237645876871561216,168,0.077022,-0.986582,0.030046,0.022980,19.460522,0.025044,4.412906e+09,23.513395,0.712944,4.412906e+09,21.640930,0.072776,4.412906e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250919,1237680512016252950,1237680512016252928,39,0.081647,34.243627,0.100617,0.107357,21.217854,0.111191,4.765158e+09,23.961412,1.420360,4.765158e+09,23.467527,0.340532,4.765159e+09
250920,1237680512016253546,1237680512016252928,39,0.081656,34.130768,0.053586,0.063756,19.941610,0.036617,4.765158e+09,26.169519,0.759941,4.765158e+09,22.433836,0.138546,4.765159e+09
250921,1237680512016253987,1237680512016252928,39,0.081701,34.083480,0.133191,0.145758,21.581175,0.159961,4.765158e+09,23.093163,0.831049,4.765158e+09,23.263212,0.287414,4.765159e+09
250922,1237680512553124114,1237680512553123840,39,0.081744,34.664146,0.008183,0.008556,18.027143,0.008862,4.765158e+09,25.195864,1.380749,4.765158e+09,19.822149,0.017225,4.765159e+09


In [35]:
raw_data.columns

Index(['objID', 'fieldID', 'field', 'ra', 'dec', 'raErr', 'decErr', 'i',
       'err_i', 'TAI_i', 'u', 'err_u', 'TAI_u', 'g', 'err_g', 'TAI_g'],
      dtype='object')

In [39]:
from astropy.time import Time

filters = ['u', 'g', 'i']
columns = ['ra', 'dec', 'fieldID', 'objID', 'raErr', 'decErr', 'obs']
df_list = []
for f in filters:
    df = pd.DataFrame(columns=['obs_id', 'ra', 'dec', 'ra_sigma', 'dec_sigma', 'mag', 'mag_sigma', 'filter', 'observatory_code'])
    
    df['obs_id'] = raw_data['objID'].apply(lambda x: f'{x}_{f}')
    df['exposure_id'] = raw_data['fieldID']
    df['ra'] = raw_data['ra']
    df['ra_sigma'] = raw_data['raErr']
    df['dec'] = raw_data['dec']
    df['dec_sigma'] = raw_data['decErr']
    df['filter'] = f
    df['mag'] = raw_data[f]
    df['mag_sigma'] = raw_data[f'err_{f}']
    df['mjd_utc'] = Time(raw_data[f'TAI_{f}']/(24*3600), scale='tai', format='mjd').utc.mjd
    df['observatory_code'] = '645'
    df_list.append(df)
    
df = pd.concat(df_list, ignore_index=True)
df.sort_values(by=['mjd_utc', 'exposure_id'], ignore_index=True)

,obs_id,ra,dec,ra_sigma,dec_sigma,mag,mag_sigma,filter,observatory_code,exposure_id,mjd_utc
0,1237645876871561298_i,0.000245,-1.034296,0.002277,0.001837,16.290249,0.004063,i,645,1237645876871561216,51075.298600
1,1237645876871561300_i,0.000173,-0.838730,0.006489,0.005588,18.285872,0.010376,i,645,1237645876871561216,51075.298600
2,1237645876871561491_i,0.000226,-0.884496,0.039081,0.035090,19.574667,0.033383,i,645,1237645876871561216,51075.298600
3,1237645876871561233_i,0.030843,-0.906976,0.066555,0.083157,20.235672,0.049130,i,645,1237645876871561216,51075.298715
4,1237645876871561348_i,0.030787,-0.855554,0.015622,0.013614,18.512777,0.012075,i,645,1237645876871561216,51075.298715
...,...,...,...,...,...,...,...,...,...,...,...
752767,1237680514163737557_g,0.090369,35.842972,0.113184,0.138118,24.643091,0.682343,g,645,1237680514163736576,55152.297986
752768,1237680514163737566_g,0.095524,35.933415,0.063354,0.079299,23.246637,0.280273,g,645,1237680514163736576,55152.297986
752769,1237680514163736724_g,0.081007,35.875682,0.003003,0.002971,17.761066,0.005441,g,645,1237680514163736576,55152.297986
752770,1237680514163737037_g,0.080519,35.790231,0.034364,0.036788,19.106573,0.015812,g,645,1237680514163736576,55152.297986


In [65]:
ras = np.linspace(0, 360, 360 * 10 + 1)
for i in range(len(ras)-1):
    query_by_ra(ras[i], ras[i+1], check_download_integrity=False)

55.0 55.1


KeyboardInterrupt: 

In [9]:
#times taken from casjobs query
TAI_i_min = 4412900195.78825
TAI_i_max = 4765233246.05795

mjd_min = TAI_i_min/(24*3600)
mjd_max = TAI_i_max/(24*3600)

mjd_min, mjd_max

(51075.23374754919, 55153.162570115164)

In [77]:
min_max = SDSS.query_sql_async("SELECT min(TAI_i) as min_TAI_i, max(TAI_i) as max_TAI_i FROM PhotoObj WHERE TAI_i != -9999", 
               timeout = 3600,
               data_release=9)

"""SELECT
    team_id,
    MAX(salary)
    FROM
    workers
    GROUP BY team_id
"""
min_max

<Response [200]>

In [78]:
min_max["min_TAI_i"]

TypeError: 'Response' object is not subscriptable

In [10]:
import glob
import multiprocessing as mp
from astropy.time import Time

window_size = 31
window_starts = np.arange(
    np.floor(mjd_min), 
    np.ceil(mjd_max), 
    window_size
)

observation_files = glob.glob(os.path.join(DATA_DIR, '*'))

In [11]:
def processWindow(window_file_name, observations):
    if len(observations) > 0:
        observations.to_hdf(
            window_file_name, 
            key="data", 
            mode="a", 
            append=True, 
            min_itemsize={'obs_id': 40, 'exposure_id': 40, 'filter' : 1, 'observatory_code': 1},
        )
    return

In [61]:
#joachim's code - sort csv file entries into months and save them as hdf5 files - windows

def convert_to_hdf5(observation_files, test=False):
    os.nice(10)

    pool = mp.Pool(10)

    observation_files_completed = np.array([])
    for i, observation_file in enumerate(observation_files):
        observations = pd.read_csv(observation_file, index_col=False, 
                                   dtype={'observatory_code': str, 'obs_id': str, 'exposure_id': str})

        windows = []
        window_file_names = []
        for window_start in window_starts:

            window_end = window_start + window_size
            start_isot = Time(window_start, scale="utc", format="mjd").isot.split("T")[0]
            end_isot = Time(window_end, scale="utc", format="mjd").isot.split("T")[0]

            if test:
                window_file_name = os.path.join(DATA_DIR, "hdf5_test", f"sdss_dr9_observations_{start_isot}_{end_isot}.h5")
            else:
                window_file_name = os.path.join(DATA_DIR, "hdf5", f"sdss_dr9_observations_{start_isot}_{end_isot}.h5")
            window_file_names.append(window_file_name)

            observations_window = observations[(observations["mjd_utc"] >= window_start) & (observations["mjd_utc"] < window_end)]
            windows.append(observations_window)

        pool.starmap(
            processWindow,
            zip(window_file_names, windows)
        )

        observation_files_completed = np.concatenate([observation_files_completed, np.array([observation_file])])
        np.savetxt("files_processed.txt", observation_files_completed, delimiter="\n", fmt="%s")

        if (i + 1) % 20 == 0:
            print(f"Processed {i + 1} observations files.")

    pool.close()

In [62]:
OLD_DATA_DIR = '/astro/store/epyc3/data3/adam_datasets/sdss_dr9'
sample_obs_file = [os.path.join(DATA_DIR, f"sdss_dr9_observations_{ra_start:06.2f}_{ra_stop:06.2f}_all.csv")]
convert_to_hdf5(sample_obs_file, test=True)

ValueError: invalid combination of [values_axes] on appending data [name->obs_id,cname->obs_id,dtype->bytes320,kind->string,shape->(1, 18400)] vs current table [name->obs_id,cname->obs_id,dtype->int64,kind->integer,shape->None]

In [24]:
TEST_DIR = '/astro/store/epyc3/data3/adam_datasets/sdss_dr9/test'
#obs_files = glob.glob(os.path.join(DATA_DIR, '*'))
test_obs_files = glob.glob(os.path.join(TEST_DIR, '*'))
convert_to_hdf5(test_obs_files, test=True)

Processed 20 observations files.


In [8]:
#run index_observations py script (slide 7 of precovery)
#bug in precovery/sourcecatalog.py - line 78 - must convert to string to use encode()
from astropy.time import Time
t = Time(2459261.291142335476, format='jd', scale='tdb')
t.tt.mjd

59260.79114232246

In [1]:
from precovery.orbit import Orbit, EpochTimescale
from precovery import precover

orbit = Orbit.keplerian(
    2202930,
    2.719500955919493,
    0.1298610598232667,
    4.562907166251742,
    346.1459277333923,
    7.61979564713161,
    118.5024332609956,
    59260.79114232246,
    EpochTimescale.TT, #data is in scale TDB (barycentric dynamical time)
    16.82,
    .15 #photometric slope parameter - what to put in?
)

orbit.compute_ephemeris('645', [51075.299190])

645
[51075.29919]


In [2]:
from precovery.orbit import Orbit, EpochTimescale
from precovery import precover

DB_DIR = "/astro/store/epyc3/data3/adam_datasets/sdss_dr9/precovery_results_2"

orbit = Orbit.keplerian(
    2202930,
    2.719500955919493,
    0.1298610598232667,
    4.562907166251742,
    346.1459277333923,
    7.61979564713161,
    118.5024332609956,
    59260.79114232246,
    EpochTimescale.TT, #data is in scale TDB (barycentric dynamical time)
    16.82,
    .15 #photometric slope parameter - what to put in?
)


results = precover(orbit, DB_DIR, tolerance=1/3600)

b'\x85\x02\x00\x00\x00\x00\x00\x00'
[51075.29895833333, 51082.29895833333, 51460.29895833333, 51467.29895833333, 51789.29895833333, 51810.29895833333, 51817.29895833333, 51866.29895833333, 51880.29895833333, 52146.29895833333, 52167.29895833333, 52195.29895833333, 52223.29895833333, 52230.29895833333, 52251.29895833333, 52524.29895833333, 52552.29895833333, 52559.29895833333, 52580.29895833333, 52587.29895833333, 52909.29895833333, 52930.29895833333, 52937.29895833333, 52958.29895833333, 52965.29895833333, 53259.29895833333, 53266.29895833333, 53273.29895833333, 53287.29895833333, 53294.29895833333, 53315.29895833333, 53350.29895833333, 53623.29895833333, 53637.29895833333, 53644.29895833333, 53994.29895833333, 54008.29895833333, 54715.29895833333, 54743.29895833333, 54750.29895833333, 54764.29895833333, 54771.29895833333, 54792.29895833333, 54827.29895833333, 54848.29895833333, 54855.29895833333, 54862.29895833333, 55100.29895833333, 55121.29895833333, 55149.29895833333]


***ERROR*** 20 Sep 2022 22:18:21UTC (Observatories / getPosition) Code b'\x85\x02\x00\x00\x00\x00\x00\x00' does not refer to any observatory listed in /astro/users/ejgl/.conda/envs/precovery_py39/share/openorb/OBSCODE.dat
***ERROR*** 20 Sep 2022 22:18:21UTC (Observatories / getGeocentricObservatoryCCoord) TRACE BACK (15)
***ERROR*** 20 Sep 2022 22:18:21UTC (Observatories / getObservatoryCCoord) TRACE BACK (15)


AssertionError: Error: 36

In [4]:
#import sqlite3 as sql

#con = sql.connect(DB_DIR + '/index.db')

In [5]:
#con.execute('CREATE index fast_query on frames (mjd, healpixel, obscode)')

In [7]:
#frames = pd.read_sql('SELECT * FROM frames', con)

In [8]:
#frames

,id,dataset_id,obscode,exposure_id,filter,mjd,healpixel,data_uri,data_offset,data_length
0,1,SDSS_DR9,b'\x85\x02\x00\x00\x00\x00\x00\x00',b'\x00\x00\xa8 ^\x00-\x11',i,51075.299190,4351,frames_00000000.data,0,355
1,2,SDSS_DR9,b'\x85\x02\x00\x00\x00\x00\x00\x00',b'\x00\x00\xa8@^\x00-\x11',i,51075.299190,4351,frames_00000000.data,355,355
2,3,SDSS_DR9,b'\x85\x02\x00\x00\x00\x00\x00\x00',b'\x00\x00\xa8`^\x00-\x11',i,51075.299190,4351,frames_00000000.data,710,568
3,4,SDSS_DR9,b'\x85\x02\x00\x00\x00\x00\x00\x00',b'\x00\x00\xa8`^\x00-\x11',i,51075.299190,4522,frames_00000000.data,1278,71
4,5,SDSS_DR9,b'\x85\x02\x00\x00\x00\x00\x00\x00',b'\x00\x00\xa8\x80^\x00-\x11',i,51075.299190,4864,frames_00000000.data,1349,213
...,...,...,...,...,...,...,...,...,...,...
2356951,2356952,SDSS_DR9,b'\x85\x02\x00\x00\x00\x00\x00\x00',b'\x00\x005@\xde\x1f-\x11',i,55152.301620,5108,frames_00000000.data,468735610,71
2356952,2356953,SDSS_DR9,b'\x85\x02\x00\x00\x00\x00\x00\x00',b'\x00\x005`\xde\x1f-\x11',i,55152.301620,5108,frames_00000000.data,468735681,142
2356953,2356954,SDSS_DR9,b'\x85\x02\x00\x00\x00\x00\x00\x00',b'\x00\x005\x80\xde\x1f-\x11',i,55152.301620,5108,frames_00000000.data,468735823,71
2356954,2356955,SDSS_DR9,b'\x85\x02\x00\x00\x00\x00\x00\x00',b'\x00\x005\xa0\xde\x1f-\x11',i,55152.301620,5108,frames_00000000.data,468735894,71


In [19]:
hd_df = pd.read_hdf('/astro/users/ejgl/epyc/data3/adam_datasets/sdss_dr9/hdf5/sdss_dr9_observations_1998-09-19_1998-10-20.h5')
hd_df

,obs_id,exposure_id,mjd_utc,ra,dec,ra_sigma,dec_sigma,filter,mag,mag_sigma,observatory_code
0,1237645876871561542,1237645876871561216,51075.299190,0.044252,-0.937640,0.146648,0.136783,i,20.067837,0.081154,645
1,1237645876871561543,1237645876871561216,51075.299190,0.045429,-0.872684,0.065231,0.057989,i,20.505260,0.063039,645
2,1237645876871561544,1237645876871561216,51075.299190,0.045675,-0.949524,0.156135,0.097594,i,21.049526,0.125765,645
3,1237645876871561715,1237645876871561216,51075.299190,0.044419,-0.998018,0.096438,0.063111,i,20.161160,0.067760,645
4,1237645876871561884,1237645876871561216,51075.299190,0.045444,-0.978974,0.300121,0.218308,i,21.821657,0.324532,645
...,...,...,...,...,...,...,...,...,...,...,...
240726,1237645879557095775,1237645879557095424,51075.306481,2.673712,1.117745,0.317605,0.181244,i,21.995596,0.341091,645
240727,1237646010010567116,1237646010010566656,51081.290046,2.673732,-1.061100,0.104356,0.074070,i,20.147598,0.068207,645
240728,1237646011084309063,1237646011084308480,51081.290046,2.673707,-0.287716,0.107323,0.133157,i,21.026785,0.105682,645
240729,1237646011621180043,1237646011621179392,51081.290046,2.673600,0.183348,0.219226,0.174383,i,21.472067,0.164109,645


In [20]:
# run for observatory_code, exposure_id, observation_id
hd_df['observatory_code'] = hd_df['observatory_code'].astype(str)
hd_df['exposure_id'] = hd_df['exposure_id'].astype(str)
hd_df['obs_id'] = hd_df['obs_id'].astype(str)

In [23]:
HDF_DIR = '/astro/users/ejgl/epyc/data3/adam_datasets/sdss_dr9/hdf5'

hdf5_files = glob.glob(os.path.join(HDF_DIR, '*'))
for file in hdf5_files:
    hd_df = pd.read_hdf(file)
    hd_df['observatory_code'] = hd_df['observatory_code'].astype(str)
    hd_df['exposure_id'] = hd_df['exposure_id'].astype(str)
    hd_df['obs_id'] = hd_df['obs_id'].astype(str)
    hd_df.to_hdf(file, key='data')

In [28]:
hd_df = pd.read_hdf('/astro/users/ejgl/epyc/data3/adam_datasets/sdss_dr9/hdf5/sdss_dr9_observations_1998-09-19_1998-10-20.h5')
hd_df['obs_id'].dtype

dtype('O')

In [18]:
con.close()